In [0]:
# from google.colab import files
# uploaded = files.upload()


In [0]:
print('Importing Libraries')
import pandas as pd
import numpy as np

print('Importing data')
df = pd.read_csv('data.csv',index_col=0)

df['shot_id_number'] = df['shot_id_number'].interpolate()
df = df.reset_index().set_index('shot_id_number')

show=0
oha=0

Importing Libraries
Importing data


In [0]:
if show:
    for col in df.columns:
        print(len(df[col].unique()),end='\t')
        print(col)

In [0]:
print('Pre-Processing')
df=df.drop('index',1)
# single value-ed columns
df=df.drop('team_name',1)
df=df.drop('team_id',1)

#splitting
df[['lat','lng']] = df['lat/lng'].str.split(', ',expand=True)
df['lat']=pd.to_numeric(df['lat'], errors='coerce')
df['lng']=pd.to_numeric(df['lng'], errors='coerce')
df.drop('lat/lng',1,inplace=True)

# repeated and converting duration to year
df=df.drop('date_of_game',1)
df['game_season'] = df['game_season'].str.replace('-\d+','')
df['game_season']=pd.to_numeric(df['game_season'])
df['game_season']-=min(df['game_season'])

# remove "MANU {@,vs.}""
df['home/away'] = df['home/away'].str.replace('[A-Z]* @ ','',regex=True)
df['home/away'] = df['home/away'].str.replace('[A-Z]* vs. ','',regex=True)

# remove "shot-" and merge type_of_shot and type_of_combined_shot
df['type_of_shot'] = df['type_of_shot'].str.replace('.*- ','')
df['type_of_combined_shot'] = df['type_of_combined_shot'].str.replace('.*- ','')
df=df.rename(columns={'type_of_combined_shot': 'type_of_shot.1'})

#removing duplicate columns
print('Removing duplicate columns')
for col in ['remaining_min','remaining_sec','power_of_shot','knockout_match','distance_of_shot']:
    df[col] = df.apply(lambda row: row[col+'.1'] if (row[col]==np.nan) and isinstance(row[col+'.1'],type(1)) else row[col],axis=1)
    df[col]=pd.to_numeric(df[col])
    df.drop(col+'.1',1,inplace=True)


Pre-Processing
Removing duplicate columns


In [0]:
if show:
    import seaborn as sns
    sns.set(rc={'figure.figsize':(15,15)})
    corr = df.corr()
    sns.heatmap(corr, 
                xticklabels=corr.columns,
                annot=True,
                cmap='Blues')


In [0]:
# checking correlation
if show:
    s = (df.dtypes == 'object')
    object_cols = list(s[s].index)
    dff=df.copy()
    for col in object_cols:
        dff[col]=pd.factorize(df[col])[0]

    corr1=dff.corr()
    c=(corr1['is_goal'])
    c=c.sort_values()
    order = c.map(lambda x : x).abs().sort_values(ascending = False)
    print(order)


In [0]:
print('Useless columns removal')
df.drop('match_id',1,inplace=True)
df.drop('lng',1,inplace=True)
df.drop('lat',1,inplace=True)
df.drop('knockout_match',1,inplace=True)

df.drop('home/away',1,inplace=True)
df.drop('location_x',1,inplace=True)
df.drop('match_event_id',1,inplace=True)


Useless columns removal


In [0]:
# after cleaning a bit
if show:
    import seaborn as sns
    sns.set(rc={'figure.figsize':(15,15)})
    corr = df.corr()
    sns.heatmap(corr, 
                xticklabels=corr.columns,
                annot=True,
                cmap='Blues')


In [0]:
# can remove
# checking correlation
if show:
    s = (df.dtypes == 'object')
    object_cols = list(s[s].index)
    dff=df.copy()
    for col in object_cols:
        dff[col]=pd.factorize(df[col])[0]

    corr1=dff.corr()
    c=(corr1['is_goal'])
    c=c.sort_values()
    order = c.map(lambda x : x).abs().sort_values(ascending = False)
    print(order)


In [0]:
# checking correlation between range_of_shot and shot_basics, area_of_shot, location_y
if show:
    object_cols = ['range_of_shot', 'shot_basics', 'area_of_shot', 'location_y','distance_of_shot']
    dff=df[object_cols]
    for col in object_cols:
        dff[col]=pd.factorize(df[col])[0]

    sns.set(rc={'figure.figsize':(5,5)})
    corr = dff.corr()
    sns.heatmap(corr, 
                xticklabels=corr.columns,
                annot=True,
                cmap='Blues')


In [0]:
# checking correlation between range_of_shot and shot_basics, area_of_shot, location_y
if show:
    s = (df.dtypes == 'object')
    object_cols = list(s[s].index)
    dff=df.copy()
    for col in object_cols:
        dff[col]=pd.factorize(df[col])[0]

    corr1=dff.corr()
    c=(corr1['distance_of_shot'])
    c=c.sort_values()
    order = c.map(lambda x : x).abs().sort_values(ascending = False)
    print(order)


In [0]:
# correlation between all
if show:
    s = (df.dtypes == 'object')
    object_cols = list(s[s].index)
    dff=df.copy()
    for col in object_cols:
        dff[col]=pd.factorize(df[col])[0]

    sns.set(rc={'figure.figsize':(12,12)})
    corr = dff.corr()
    sns.heatmap(corr, 
                xticklabels=corr.columns,
                annot=True,
                cmap='Blues')


In [0]:
# correlation between is_goal and all
if show:
    s = (df.dtypes == 'object')
    object_cols = list(s[s].index)
    dff=df.copy()
    for col in object_cols:
        dff[col]=pd.factorize(df[col])[0]

    corr1=dff.corr()
    c=(corr1['is_goal'])
    c=c.sort_values()
    order = c.map(lambda x : x).abs().sort_values(ascending = False)
    print(order)


In [0]:
# onehot encoding
# df one hot all
if oha:
    print('Populating dummies')

#     from sklearn.preprocessing import LabelEncoder
    s = (df.dtypes == 'object')
    object_cols = list(s[s].index)
    df = pd.get_dummies(df, columns=object_cols, prefix = object_cols)
else:
    print('Useless "distance_of_shot" correlated removal')
    df.drop('shot_basics',1,inplace=True)
    df.drop('area_of_shot',1,inplace=True)
    df.drop('location_y',1,inplace=True)
    df.drop('range_of_shot',1,inplace=True)
    
cols = df.columns.tolist()
cols = cols[:cols.index('is_goal')] + cols[cols.index('is_goal')+1:]
cols.append('is_goal')
df = df[cols]
df.columns


Useless "distance_of_shot" correlated removal


Index(['remaining_min', 'power_of_shot', 'game_season', 'remaining_sec',
       'distance_of_shot', 'type_of_shot', 'type_of_shot.1', 'is_goal'],
      dtype='object')

In [0]:
# from sklearn import preprocessing
# def normalize(dataset):
#     dataNorm=((dataset-dataset.min())/(dataset.max()-dataset.min()))*20
#     dataNorm["is_goal"]=dataset["is_goal"]
#     return dataNorm

# df=normalize(df)



TypeError: ignored

In [0]:
if 0:
    sns.jointplot(x=pd.factorize(df['range_of_shot'])[0], y=pd.factorize(df['shot_basics'])[0], kind="kde")
    sns.jointplot(x=pd.factorize(df['range_of_shot'])[0], y=df['distance_of_shot'], kind="kde")
    sns.jointplot(x=pd.factorize(df['range_of_shot'])[0], y=pd.factorize(df['area_of_shot'])[0], kind="kde")
    sns.jointplot(x=pd.factorize(df['range_of_shot'])[0], y=(df['location_y']), kind="kde")


In [0]:
print('Dealing with NaN')
# checkpoint
dfsub=df.copy()

df=df.fillna(df.mean())
# dfr=df.dropna(axis=0)
# print(len(dfr))

In [0]:
print('y,X')
df=df[df['is_goal'].notnull()]
y=df.is_goal
X=df.loc[:,df.columns!='is_goal']

from sklearn.model_selection import train_test_split
print('Train-Test split')
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)


In [0]:
print('Decision Tree Regressor')
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor()
print('Training')
model.fit(train_X, train_y)
print('Testing')
val_predictions = model.predict(val_X)

from sklearn.metrics import mean_absolute_error
print("MAE:",mean_absolute_error(val_y, val_predictions))

val_predictions = model.predict(train_X)
print("OMAE:",mean_absolute_error(train_y, val_predictions))


In [0]:
print('Random Forest Regressor')
from sklearn.ensemble import RandomForestRegressor
forest_model = RandomForestRegressor(n_estimators=100)
print('Training')
forest_model.fit(train_X, train_y)
print('Testing')
val_predictions = forest_model.predict(val_X)

from sklearn.metrics import mean_absolute_error
print("MAE:",mean_absolute_error(val_y, val_predictions))

val_predictions = forest_model.predict(train_X)
print("OMAE:",mean_absolute_error(train_y, val_predictions))


In [0]:
print('XGBoost')
from xgboost import XGBRegressor

XG = XGBRegressor(objective='reg:squarederror')

print('Training')
XG.fit(train_X, train_y)
print('Testing')
val_predictions = XG.predict(val_X)

from sklearn.metrics import mean_absolute_error
print("MAE:",mean_absolute_error(val_y, val_predictions))

val_predictions = XG.predict(train_X)
print("OMAE:",mean_absolute_error(train_y, val_predictions))



In [0]:
print('SUBMISSION')
print('y,X')
df=dfsub
y=df.is_goal
X=df.loc[:,df.columns!='is_goal']

print('Seperating data_set')
dfqnt=df[df['is_goal'].notnull()]
dfans=df[df['is_goal'].isnull()]

print('Train-Test split')

train_X = dfqnt.loc[:,df.columns!='is_goal']
train_X=train_X.fillna(df.mean())

train_y=dfqnt.is_goal

val_X = dfans.loc[:,df.columns!='is_goal']
val_X=val_X.fillna(df.mean())

# val_y = dfans.is_goal


In [0]:
print('XGBoost')
from xgboost import XGBRegressor
XG = XGBRegressor()

print('Training')
XG.fit(train_X, train_y)
print('Testing')
val_predictions = XG.predict(val_X)

to_save_df=pd.DataFrame(columns=['shot_id_number','is_goal'])
to_save_df['shot_id_number']=val_X.index
to_save_df['is_goal']=pd.DataFrame(val_predictions)
to_save_df.to_csv('submission.csv',index=False)


In [0]:
# ranking of features
model=XG
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

print("Feature ranking:")
for f in range(X.shape[1]):
      print("%d. %s (%f)" % (f + 1, df.columns[indices[f]], importances[indices[f]]))


In [0]:
importances = model.feature_importances_
indices = np.argsort(importances)
import matplotlib.pyplot as plt

plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [df.columns[i] for i in indices])
plt.xlabel('Relative Importance')
plt.figure(figsize=(12,12))
plt.show()
